## Imports

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import sklearn

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

from pathlib import Path
from IPython.display import Image, display, Video, HTML
from ipywidgets import interact, widgets

from signlens.params import *
from signlens.preprocessing import data, preprocess
from utils import plot_landmarks

# reload automatically python functions outside notebook
%load_ext autoreload
%autoreload 2

2024-03-20 17:27:01.914641: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-20 17:27:04.694352: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Fetch data

In [11]:
DATA_FRAC

In [8]:
NUM_CLASSES
MAX_SEQ_LEN
DATA_FRAC

In [2]:
NUM_SIGNS = 10
MAX_SEQ_LEN = 100
FRAC_DATA = 0.01

In [36]:
sequences_data = data.load_data_subset_csv(noface=True, frac=FRAC_DATA, balanced=True, num_signs=NUM_CLASSES)
sequences_data = data.load_frame_number_parquet(sequences_data)
sequences_data = preprocess.filter_out_parquet_frame(sequences_data, MAX_SEQ_LEN)
sequences_data.head()

Size reduced from 94477 to 944 (1.0%)
File already exists, loaded matching 'sequence_id' rows.


,path,participant_id,sequence_id,sign,file_path,frame_parquet
0,train_landmark_files_noface/22343/1000638205.p...,22343,1000638205,puzzle,../../raw_data/asl-signs/train_landmark_files_...,19.0
1,train_landmark_files_noface/32319/1001958254.p...,32319,1001958254,go,../../raw_data/asl-signs/train_landmark_files_...,24.0
2,train_landmark_files_noface/2044/1004782423.pa...,2044,1004782423,look,../../raw_data/asl-signs/train_landmark_files_...,24.0
3,train_landmark_files_noface/36257/1030751383.p...,36257,1030751383,tongue,../../raw_data/asl-signs/train_landmark_files_...,12.0
4,train_landmark_files_noface/49445/104061906.pa...,49445,104061906,puzzle,../../raw_data/asl-signs/train_landmark_files_...,48.0


In [60]:
sequences_data.sign.value_counts()

sign
on          92
thirsty     90
scissors    89
go          88
look        88
tongue      88
owl         86
green       85
rain        82
puzzle      77
Name: count, dtype: int64

In [46]:
y =  preprocess.label_dictionnary_not_encoded(sequences_data) # encode the labels to numbers
y

0      180
1       97
2      136
3      222
4      180
      ... 
860    160
861    102
862    222
863    160
864     97
Name: sign_encoded, Length: 865, dtype: int64

In [61]:
y.shape


(865, 10)

In [62]:
from sklearn.model_selection import train_test_split

X_files = sequences_data.file_path
y = preprocess.label_dictionnary(sequences_data) # not very clean... needs to be iomrpoved

# Train test split
X_train_files, X_test_files, y_train, y_test = train_test_split(X_files, y, test_size=0.3, stratify=y)

# Train split into train and val
X_train_files, X_val_files, y_train, y_val = train_test_split(X_train_files, y_train, test_size=0.3, stratify=y_train)

In [73]:
X_train = preprocess.group_pad_sequences(pd.DataFrame(X_train_files), frame=MAX_SEQ_LEN)
X_val = preprocess.group_pad_sequences(pd.DataFrame(X_val_files), frame=MAX_SEQ_LEN)
X_test = preprocess.group_pad_sequences(pd.DataFrame(X_test_files), frame=MAX_SEQ_LEN)




In [65]:
# X_train = X_train.reshape(*X_train.shape[:-2], -1)
# X_val = X_val.reshape(*X_val.shape[:-2], -1)
# X_test = X_test.reshape(*X_test.shape[:-2], -1)


In [66]:
X_train.shape

(423, 100, 225)

## Model

In [72]:
y.shape

(865, 10)

In [67]:
from tensorflow.keras import Model, Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import models, layers, regularizers

from tensorflow.keras.utils import to_categorical

In [69]:
# 1- RNN Architecture
model = Sequential()
model.add(Reshape((MAX_SEQ_LEN, N_LANDMARKS_NO_FACE*3), input_shape=(MAX_SEQ_LEN, N_LANDMARKS_NO_FACE, 3)))
model.add(layers.SimpleRNN(units=40, activation='tanh', input_shape=(MAX_SEQ_LEN, N_LANDMARKS_NO_FACE * 3)))
model.add(layers.Dense(20, activation="linear"))
model.add(layers.Dense(20, activation="linear"))
model.add(layers.Dense(NUM_SIGNS, activation='softmax'))


y_cat = to_categorical(y)

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

es = EarlyStopping(patience=10, restore_best_weights=True)

history = model.fit(X_train, y_train,
				validation_data=(X_val, y_val),
			    epochs=10,
			    batch_size=32, 
			    verbose=1, 
			    callbacks = [es] # This will call the Early Stopping Criterion for each epoch
			)

Epoch 1/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 5s 191ms/step - accuracy: 0.0822 - loss: 0.6101 - val_accuracy: 0.1044 - val_loss: 0.3595
Epoch 2/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 81ms/step - accuracy: 0.0806 - loss: 0.3500 - val_accuracy: 0.1044 - val_loss: 0.3322
Epoch 3/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 93ms/step - accuracy: 0.0842 - loss: 0.3303 - val_accuracy: 0.1154 - val_loss: 0.3281
Epoch 4/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 81ms/step - accuracy: 0.1020 - loss: 0.3271 - val_accuracy: 0.1099 - val_loss: 0.3278
Epoch 5/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 81ms/step - accuracy: 0.0931 - loss: 0.3264 - val_accuracy: 0.1099 - val_loss: 0.3257
Epoch 6/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 89ms/step - accuracy: 0.1171 - loss: 0.3260 - val_accuracy: 0.1044 - val_loss: 0.3272
Epoch 7/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 93ms/step - accuracy: 0.0897 - loss: 0.3271 - val_accuracy: 0.1044 - val_loss: 0.3268
Epoch 8/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 101ms/step - accuracy: 0.1184 - loss: 0.3260 - val_accuracy: 0.1044 -

In [ ]:
model = Sequential()
model.add(TimeDistributed(Flatten(), input_shape=(MAX_SEQ_LEN, N_LANDMARKS_NO_FACE, 3)))
model.add(LSTM(units=128))
model.add(Dense(num_classes, activation='softmax'))